In [1]:
# =============================================================================
# BLOCK 1: SETUP, IMPORTS, AND DATA LOADING
# =============================================================================
import warnings
warnings.filterwarnings('ignore')
import time
import os
# --- Library Imports ---
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import catboost as cb
import optuna
print("Libraries imported successfully.")
# --- Helper Function for Winkler Score ---
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):
    width = upper - lower
    penalty_lower = np.where(y_true < lower, (2 / alpha) * (lower - y_true), 0)
    penalty_upper = np.where(y_true > upper, (2 / alpha) * (y_true - upper), 0)
    score = width + penalty_lower + penalty_upper
    if return_coverage:
        coverage = np.mean((y_true >= lower) & (y_true <= upper))
        return np.mean(score), coverage
    return np.mean(score)
# --- Global Constants ---
N_SPLITS = 5
RANDOM_STATE = 42
DATA_PATH = './'
N_OPTUNA_TRIALS = 75 # A strong number for a comprehensive search
COMPETITION_ALPHA = 0.1

# --- Load Raw Data ---
try:
    # We drop the low-variance columns they identified right away
    drop_cols=['id', 'golf', 'view_rainier', 'view_skyline', 'view_lakesamm','view_otherwater', 'view_other']
    df_train = pd.read_csv(DATA_PATH + 'dataset.csv').drop(columns=drop_cols)
    df_test = pd.read_csv(DATA_PATH + 'test.csv').drop(columns=drop_cols)
    print("Raw data loaded successfully.")
except FileNotFoundError:
    print("ERROR: Could not find 'dataset.csv' or 'test.csv'.")
    exit()
# --- Prepare Target Variable ---
y_true = df_train['sale_price'].copy()
grade_for_stratify = df_train['grade'].copy()
# The mean-error model works best when predicting the raw price directly
# So, we will NOT log-transform the target this time.
# df_train.drop('sale_price', axis=1, inplace=True) # We keep sale_price for FE
print("Setup complete.")


Libraries imported successfully.
Raw data loaded successfully.
Setup complete.


In [2]:
# Make sure to have these libraries installed
# pip install pandas numpy scikit-learn

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
import gc

# Define a random state for reproducibility
RANDOM_STATE = 42

def create_comprehensive_features(df_train, df_test):
    """
    Combines original and new advanced feature engineering steps into a single pipeline.
    """
    print("--- Starting Comprehensive Feature Engineering ---")

    # Store original indices and target variable
    train_ids = df_train.index
    test_ids = df_test.index
    y_train = df_train['sale_price'].copy() # Keep the target separate

    # Combine for consistent processing
    df_train_temp = df_train.drop(columns=['sale_price'])
    all_data = pd.concat([df_train_temp, df_test], axis=0, ignore_index=True)

    # --- Original Feature Engineering ---

    # A) Brute-Force Numerical Interactions
    print("Step 1: Creating brute-force numerical interaction features...")
    NUMS = ['area', 'land_val', 'imp_val', 'sqft_lot', 'sqft', 'sqft_1', 'grade', 'year_built']
    # Ensure all columns exist and are numeric, fill missing with 0 for safety
    for col in NUMS:
        if col not in all_data.columns:
            all_data[col] = 0
        else:
            all_data[col] = pd.to_numeric(all_data[col], errors='coerce').fillna(0)
            
    for i in range(len(NUMS)):
        for j in range(i + 1, len(NUMS)):
            all_data[f'{NUMS[i]}_x_{NUMS[j]}'] = all_data[NUMS[i]] * all_data[NUMS[j]]

    # B) Date Features
    print("Step 2: Creating date features...")
    all_data['sale_date'] = pd.to_datetime(all_data['sale_date'])
    all_data['sale_year'] = all_data['sale_date'].dt.year
    all_data['sale_month'] = all_data['sale_date'].dt.month
    all_data['sale_dayofyear'] = all_data['sale_date'].dt.dayofyear
    all_data['age_at_sale'] = all_data['sale_year'] - all_data['year_built']

    # C) TF-IDF Text Features
    print("Step 3: Creating TF-IDF features for text columns...")
    text_cols = ['subdivision', 'zoning', 'city', 'sale_warning', 'join_status', 'submarket']
    all_data[text_cols] = all_data[text_cols].fillna('missing').astype(str)
    
    for col in text_cols:
        tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 5), max_features=128, binary=True)
        svd = TruncatedSVD(n_components=8, random_state=RANDOM_STATE)
        
        tfidf_matrix = tfidf.fit_transform(all_data[col])
        tfidf_svd = svd.fit_transform(tfidf_matrix)
        
        tfidf_df = pd.DataFrame(tfidf_svd, columns=[f'{col}_tfidf_svd_{i}' for i in range(8)])
        all_data = pd.concat([all_data, tfidf_df], axis=1)

    # D) Log transform some interaction features
    for c in ['land_val_x_imp_val', 'land_val_x_sqft', 'imp_val_x_sqft']:
        if c in all_data.columns:
            all_data[c] = np.log1p(all_data[c].fillna(0))

    # --- New Feature Engineering Ideas ---

    # F) Group-By Aggregation Features
    print("Step 4: Creating group-by aggregation features...")
    group_cols = ['submarket', 'city', 'zoning']
    num_cols_for_agg = ['grade', 'sqft', 'imp_val', 'land_val', 'age_at_sale']

    for group_col in group_cols:
        for num_col in num_cols_for_agg:
            agg_stats = all_data.groupby(group_col)[num_col].agg(['mean', 'std', 'max', 'min']).reset_index()
            agg_stats.columns = [group_col] + [f'{group_col}_{num_col}_{stat}' for stat in ['mean', 'std', 'max', 'min']]
            all_data = pd.merge(all_data, agg_stats, on=group_col, how='left')
            all_data[f'{num_col}_minus_{group_col}_mean'] = all_data[num_col] - all_data[f'{group_col}_{num_col}_mean']

    # G) Ratio Features
    print("Step 5: Creating ratio features...")
    # Add a small epsilon to prevent division by zero
    epsilon = 1e-6 
    all_data['total_val'] = all_data['imp_val'] + all_data['land_val']
    all_data['imp_val_to_land_val_ratio'] = all_data['imp_val'] / (all_data['land_val'] + epsilon)
    all_data['land_val_ratio'] = all_data['land_val'] / (all_data['total_val'] + epsilon)
    all_data['sqft_to_lot_ratio'] = all_data['sqft'] / (all_data['sqft_lot'] + epsilon)
    all_data['was_renovated'] = (all_data['year_reno'] > 0).astype(int)
    all_data['reno_age_at_sale'] = np.where(all_data['was_renovated'] == 1, all_data['sale_year'] - all_data['year_reno'], -1)

    # H) Geospatial Clustering Features
    print("Step 6: Creating geospatial clustering features...")
    coords = all_data[['latitude', 'longitude']].copy()
    coords.fillna(coords.median(), inplace=True) # Simple imputation

    # KMeans is sensitive to feature scaling, but for lat/lon it's often okay without it.
    kmeans = KMeans(n_clusters=20, random_state=RANDOM_STATE, n_init=10) 
    all_data['location_cluster'] = kmeans.fit_predict(coords)
    
    # Calculate distance to each cluster center
    cluster_centers = kmeans.cluster_centers_
    for i in range(len(cluster_centers)):
        center = cluster_centers[i]
        all_data[f'dist_to_cluster_{i}'] = np.sqrt((coords['latitude'] - center[0])**2 + (coords['longitude'] - center[1])**2)

    # --- Final Cleanup ---
    print("Step 7: Finalizing feature set...")
    cols_to_drop = ['sale_date', 'subdivision', 'zoning', 'city', 'sale_warning', 'join_status', 'submarket']
    all_data = all_data.drop(columns=cols_to_drop)

    # One-hot encode the new cluster feature
    all_data = pd.get_dummies(all_data, columns=['location_cluster'], prefix='loc_cluster')
    
    # Final check for any remaining object columns to be safe (besides index)
    object_cols = all_data.select_dtypes(include='object').columns
    if len(object_cols) > 0:
        print(f"Warning: Found unexpected object columns: {object_cols}. Dropping them.")
        all_data = all_data.drop(columns=object_cols)
        
    all_data.fillna(0, inplace=True)

    # Separate back into train and test sets
    train_len = len(train_ids)
    X = all_data.iloc[:train_len].copy()
    X_test = all_data.iloc[train_len:].copy()
    
    # Restore original indices
    X.index = train_ids
    X_test.index = test_ids
    
    # Align columns - crucial for model prediction
    X_test = X_test[X.columns]
    
    print(f"\nComprehensive FE complete. Total features: {X.shape[1]}")
    gc.collect()
    
    return X, X_test, y_train
# =============================================================================
# BLOCK 2.5: EXECUTE FEATURE ENGINEERING
# =============================================================================
print("\n--- Starting Block 2.5: Executing Feature Engineering Pipeline ---")

# This is the crucial step that was missing.
# We call the function to create our training and testing dataframes.
X, X_test, y_train = create_comprehensive_features(df_train, df_test)

# Let's verify the output
print(f"Feature engineering complete. X shape: {X.shape}, X_test shape: {X_test.shape}")
gc.collect()


--- Starting Block 2.5: Executing Feature Engineering Pipeline ---
--- Starting Comprehensive Feature Engineering ---
Step 1: Creating brute-force numerical interaction features...
Step 2: Creating date features...
Step 3: Creating TF-IDF features for text columns...
Step 4: Creating group-by aggregation features...
Step 5: Creating ratio features...
Step 6: Creating geospatial clustering features...
Step 7: Finalizing feature set...

Comprehensive FE complete. Total features: 233
Feature engineering complete. X shape: (200000, 233), X_test shape: (200000, 233)


0

In [3]:
# =============================================================================
# BLOCK 3.5: TUNE LIGHTGBM MEAN MODEL
# =============================================================================
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna

# --- 1. Prepare Data for Tuning ---
# We use the same train/validation split from the full dataset
X_train_opt, X_val_opt, y_train_opt, y_val_opt = train_test_split(
    X, y_true, test_size=0.2, random_state=RANDOM_STATE
)

# Convert data to LightGBM's Dataset format for efficiency
lgb_train_opt = lgb.Dataset(X_train_opt, y_train_opt)
lgb_val_opt = lgb.Dataset(X_val_opt, y_val_opt, reference=lgb_train_opt)

# --- 2. Define the Optuna Objective Function for LightGBM ---
def objective_lightgbm(trial):
    """
    This function takes an Optuna 'trial' object and does the following:
    1. Defines a search space for LightGBM's hyperparameters.
    2. Trains a LightGBM model with a set of hyperparameters suggested by the trial.
    3. Evaluates the model on the validation set.
    4. Returns the validation score (RMSE), which Optuna tries to minimize.
    """
    # Define the hyperparameter search space
    params = {
        'objective': 'regression_l1',
        'metric': 'rmse',
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'boosting_type': 'gbdt',
        
        # --- THE FIX IS HERE ---
        'verbosity': -1,  # Add this line to disable all warnings
        # ---------------------
        
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 3000),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 5, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.7, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7)
    }

    # The rest of the function remains the same...
    model = lgb.train(
        params=params,
        train_set=lgb_train_opt,
        valid_sets=[lgb_val_opt],
        callbacks=[lgb.early_stopping(100, verbose=False)]
    )
    preds = model.predict(X_val_opt, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_val_opt, preds))
    return rmse

# --- 3. Create and Run the Optuna Study ---
# Create a study object and specify the direction is 'minimize' (for RMSE)
study_lgbm = optuna.create_study(direction='minimize')

# Start the optimization process. 
# n_trials can be increased for a more thorough search, but this is a good start.
print("--- Starting LightGBM Hyperparameter Tuning... ---")
study_lgbm.optimize(objective_lightgbm, n_trials=N_OPTUNA_TRIALS)

# --- 4. Print the Best Results ---
print("\n--- LightGBM Tuning Complete ---")
print(f"Best trial validation RMSE: ${study_lgbm.best_value:,.2f}")
print("Best hyperparameters found for LightGBM:")

best_params_lgbm = study_lgbm.best_params
for key, value in best_params_lgbm.items():
    print(f"  '{key}': {value},")

# Store the best parameters in a dictionary for later use in the K-Fold training loop
# (The n_estimators is already included from the tuning search)

[I 2025-07-24 15:32:39,480] A new study created in memory with name: no-name-1ed9314c-8822-45b4-88dc-da215d2c9764


--- Starting LightGBM Hyperparameter Tuning... ---


[I 2025-07-24 15:33:13,243] Trial 0 finished with value: 102657.84906543046 and parameters: {'learning_rate': 0.0694158863341818, 'n_estimators': 2350, 'num_leaves': 72, 'max_depth': 9, 'lambda_l1': 1.9858992218372273, 'lambda_l2': 0.09996862401592421, 'feature_fraction': 0.9101103081177001, 'bagging_fraction': 0.9166550428530107, 'bagging_freq': 5}. Best is trial 0 with value: 102657.84906543046.
[I 2025-07-24 15:33:39,645] Trial 1 finished with value: 103187.11873057194 and parameters: {'learning_rate': 0.0628655381280119, 'n_estimators': 2532, 'num_leaves': 72, 'max_depth': 7, 'lambda_l1': 0.10842533445489826, 'lambda_l2': 0.013299799605664488, 'feature_fraction': 0.8860513122252763, 'bagging_fraction': 0.908309782389596, 'bagging_freq': 1}. Best is trial 0 with value: 102657.84906543046.
[I 2025-07-24 15:34:05,072] Trial 2 finished with value: 103098.20971078215 and parameters: {'learning_rate': 0.050330960512912686, 'n_estimators': 2949, 'num_leaves': 61, 'max_depth': 7, 'lambda_l


--- LightGBM Tuning Complete ---
Best trial validation RMSE: $101,609.73
Best hyperparameters found for LightGBM:
  'learning_rate': 0.06641957832283381,
  'n_estimators': 2777,
  'num_leaves': 98,
  'max_depth': 10,
  'lambda_l1': 1.3088673513828804,
  'lambda_l2': 0.0014231169064005797,
  'feature_fraction': 0.781741153364829,
  'bagging_fraction': 0.8580908617679706,
  'bagging_freq': 5,


In [5]:
# =============================================================================
# BLOCK 4.5: K-FOLD TRAINING & SAVING WITH OPTIMAL LIGHTGBM PARAMETERS
# =============================================================================
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gc
import os

print("\n" + "="*80)
print("--- Step 4.5: K-Fold Cross-Validation with Optimal LightGBM Hyperparameters ---")
print("="*80)


# Initialize arrays to store the predictions
oof_lgbm_preds = np.zeros(len(X))
test_lgbm_preds = np.zeros(len(X_test))

# Use StratifiedKFold for the final training, as per the winning strategy
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

# This assumes 'best_params_lgbm' exists from the Optuna tuning block
# Let's ensure 'n_estimators' is an integer, as Optuna might return a float
best_params_lgbm['n_estimators'] = int(best_params_lgbm.get('n_estimators', 2000))
# Add other fixed parameters for scikit-learn wrapper
best_params_lgbm['random_state'] = RANDOM_STATE
best_params_lgbm['n_jobs'] = -1

for fold, (train_idx, val_idx) in enumerate(skf.split(X, grade_for_stratify)):
    print(f"\n--- Training Fold {fold+1}/{N_SPLITS} ---")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y_true.iloc[train_idx], y_true.iloc[val_idx]

    # Initialize and train the model for this fold using the best parameters
    model = lgb.LGBMRegressor(**best_params_lgbm)
    
    # LGBM's scikit-learn wrapper handles early stopping within .fit()
    model.fit(X_train, y_train_fold,
              eval_set=[(X_val, y_val_fold)],
              callbacks=[lgb.early_stopping(100, verbose=False)])
    

    
    # Generate predictions for the validation set (this fold's OOF part)
    # model.best_iteration_ is available after early stopping
    oof_preds_fold = model.predict(X_val, num_iteration=model.best_iteration_)
    oof_lgbm_preds[val_idx] = np.clip(oof_preds_fold, 0, None)
    
    # Generate predictions for the test set and accumulate them
    test_lgbm_preds += np.clip(model.predict(X_test, num_iteration=model.best_iteration_), 0, None) / N_SPLITS
    
    fold_rmse = np.sqrt(mean_squared_error(y_val_fold, oof_preds_fold))
    print(f"  Fold {fold+1} Validation RMSE: ${fold_rmse:,.2f}")
    del model, X_train, X_val, y_train_fold, y_val_fold
    gc.collect()

# --- Final Evaluation and Saving ---
print("\n" + "="*80)
print("--- Step 5.5: Final Evaluation and Saving Predictions ---")
print("="*80)

final_oof_rmse = np.sqrt(mean_squared_error(y_true, oof_lgbm_preds))
print(f"Final LightGBM OOF RMSE across all {N_SPLITS} folds: ${final_oof_rmse:,.2f}")


--- Step 4.5: K-Fold Cross-Validation with Optimal LightGBM Hyperparameters ---

--- Training Fold 1/5 ---
  Fold 1 Validation RMSE: $98,328.04

--- Training Fold 2/5 ---
  Fold 2 Validation RMSE: $97,550.84

--- Training Fold 3/5 ---
  Fold 3 Validation RMSE: $98,208.78

--- Training Fold 4/5 ---
  Fold 4 Validation RMSE: $97,811.70

--- Training Fold 5/5 ---
  Fold 5 Validation RMSE: $97,511.02

--- Step 5.5: Final Evaluation and Saving Predictions ---
Final LightGBM OOF RMSE across all 5 folds: $97,882.64


In [6]:
# Define the save path and save the final prediction arrays
SAVE_PATH = './mean_models_v1/' # Saving to the same place as other mean models
os.makedirs(SAVE_PATH, exist_ok=True)
np.save(os.path.join(SAVE_PATH, 'oof_lgbm_preds.npy'), oof_lgbm_preds)
np.save(os.path.join(SAVE_PATH, 'test_lgbm_preds.npy'), test_lgbm_preds)
print("\n'oof_lgbm_preds.npy' and 'test_lgbm_preds.npy' saved successfully.")


'oof_lgbm_preds.npy' and 'test_lgbm_preds.npy' saved successfully.
